In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
#!pip install -U langchain-openai

In [3]:
from langchain_openai import ChatOpenAI

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
KEY= os.getenv("OPENAI_API_KEY")

In [6]:
llm = ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.5)

In [7]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002E067F94EB0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002E067F96350>, temperature=0.5, openai_api_key=SecretStr('**********'), openai_proxy='')

In [8]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [9]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [10]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [11]:
quiz_generation_prompt = PromptTemplate(
    input_variables = ["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [12]:
quiz_chain = LLMChain(llm=llm,prompt=quiz_generation_prompt, output_key="quiz",verbose=True)

In [13]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [14]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [15]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [16]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [17]:
file_path = r"C:/Users/the_hood/Desktop/ineuron/mcqgenerator/data.txt"

In [18]:
with open(file_path,'r') as file:
    TEXT = file.read()

In [19]:
print(TEXT)

Kali phosphoricum or the phosphate of potassium (K2HPO4) is a homeopathic medication that is dispensed in the form of small round pills or diluted solutions.

It has widespread usages in homeopathy, including its use as a tonic for nerves and the brain. A homeopathic practitioner may prescribe this medicine in multiple doses if a patient suffers from psychosomatic effects (affecting the mind and body) of longstanding stress, such as:

chronic fatigue
tension headaches
acidity
upset stomach
It could also be prescribed (after exhaustively analyzing medical history and conducting consultation sessions) in individuals who suffer from stress-related mood swings, migraines, insomnia, anxiety, and depressive episodes.

The homeopathic texts state that potassium phosphate can be prescribed in standard homeopathic doses for problems with cognition, inattention, and difficulties during concentrating. Homeopathic medications are best taken under the supervision of a doctor. This includes the home

In [20]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [21]:
NUMBER = 10
SUBJECT = "Homeopathy"
TONE = "simple"

In [22]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )

c:\Users\the_hood\Desktop\ineuron\mcqgenerator\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Kali phosphoricum or the phosphate of potassium (K2HPO4) is a homeopathic medication that is dispensed in the form of small round pills or diluted solutions.

It has widespread usages in homeopathy, including its use as a tonic for nerves and the brain. A homeopathic practitioner may prescribe this medicine in multiple doses if a patient suffers from psychosomatic effects (affecting the mind and body) of longstanding stress, such as:

chronic fatigue
tension headaches
acidity
upset stomach
It could also be prescribed (after exhaustively analyzing medical history and conducting consultation sessions) in individuals who suffer from stress-related mood swings, migraines, insomnia, anxiety, and depressive episodes.

The homeopathic texts state that potassium phosphate can be prescribed in standard homeopathic doses for problems with cognition, inattention, and difficulties during con

In [23]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:3464
Prompt Tokens:1992
Completion Tokens:1472
Total Cost:0.005932000000000001


In [24]:
response

{'text': 'Kali phosphoricum or the phosphate of potassium (K2HPO4) is a homeopathic medication that is dispensed in the form of small round pills or diluted solutions.\n\nIt has widespread usages in homeopathy, including its use as a tonic for nerves and the brain. A homeopathic practitioner may prescribe this medicine in multiple doses if a patient suffers from psychosomatic effects (affecting the mind and body) of longstanding stress, such as:\n\nchronic fatigue\ntension headaches\nacidity\nupset stomach\nIt could also be prescribed (after exhaustively analyzing medical history and conducting consultation sessions) in individuals who suffer from stress-related mood swings, migraines, insomnia, anxiety, and depressive episodes.\n\nThe homeopathic texts state that potassium phosphate can be prescribed in standard homeopathic doses for problems with cognition, inattention, and difficulties during concentrating. Homeopathic medications are best taken under the supervision of a doctor. Th

In [31]:
print(response)

{'text': 'Kali phosphoricum or the phosphate of potassium (K2HPO4) is a homeopathic medication that is dispensed in the form of small round pills or diluted solutions.\n\nIt has widespread usages in homeopathy, including its use as a tonic for nerves and the brain. A homeopathic practitioner may prescribe this medicine in multiple doses if a patient suffers from psychosomatic effects (affecting the mind and body) of longstanding stress, such as:\n\nchronic fatigue\ntension headaches\nacidity\nupset stomach\nIt could also be prescribed (after exhaustively analyzing medical history and conducting consultation sessions) in individuals who suffer from stress-related mood swings, migraines, insomnia, anxiety, and depressive episodes.\n\nThe homeopathic texts state that potassium phosphate can be prescribed in standard homeopathic doses for problems with cognition, inattention, and difficulties during concentrating. Homeopathic medications are best taken under the supervision of a doctor. Th

In [32]:
print(type(response))

<class 'dict'>


In [33]:
print(response.keys())

dict_keys(['text', 'number', 'subject', 'tone', 'response_json', 'quiz', 'review'])


In [35]:
print(response['text'])

Kali phosphoricum or the phosphate of potassium (K2HPO4) is a homeopathic medication that is dispensed in the form of small round pills or diluted solutions.

It has widespread usages in homeopathy, including its use as a tonic for nerves and the brain. A homeopathic practitioner may prescribe this medicine in multiple doses if a patient suffers from psychosomatic effects (affecting the mind and body) of longstanding stress, such as:

chronic fatigue
tension headaches
acidity
upset stomach
It could also be prescribed (after exhaustively analyzing medical history and conducting consultation sessions) in individuals who suffer from stress-related mood swings, migraines, insomnia, anxiety, and depressive episodes.

The homeopathic texts state that potassium phosphate can be prescribed in standard homeopathic doses for problems with cognition, inattention, and difficulties during concentrating. Homeopathic medications are best taken under the supervision of a doctor. This includes the home

In [36]:
print(response['number'],response['subject'],response['tone'])

10 Homeopathy simple


In [37]:
print(response['response_json'])

{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}


In [40]:
print(response['quiz'])

{
"1": {
"mcq": "What is Kali phosphoricum?",
"options": {
"a": "A homeopathic medication made from potassium phosphate",
"b": "A modern medicine for stress relief",
"c": "A type of surgical procedure",
"d": "An antibiotic drug"
},
"correct": "a"
},
"2": {
"mcq": "What is the main usage of Kali phosphoricum in homeopathy?",
"options": {
"a": "Treatment of skin conditions",
"b": "Tonic for nerves and the brain",
"c": "Treatment of heart diseases",
"d": "Treatment of respiratory infections"
},
"correct": "b"
},
"3": {
"mcq": "Which of the following conditions can Kali phosphoricum be prescribed for?",
"options": {
"a": "High blood pressure",
"b": "Chronic fatigue",
"c": "Diabetes",
"d": "Broken bones"
},
"correct": "b"
},
"4": {
"mcq": "What caution should be taken when consuming Kali phosphoricum?",
"options": {
"a": "Avoid chewing tobacco or drinking alcohol",
"b": "Take it with any type of beverage",
"c": "Consume it with high-fat foods",
"d": "Mix it with other medications"
},
"corre

In [41]:
print(response['review'])


{
"1": {
"mcq": "What is Kali phosphoricum?",
"options": {
"a": "A homeopathic medication",
"b": "A type of antibiotic",
"c": "A surgical procedure",
"d": "A vitamin supplement"
},
"correct": "a"
},
"2": {
"mcq": "What is the main usage of Kali phosphoricum in homeopathy?",
"options": {
"a": "To treat skin conditions",
"b": "To improve digestion",
"c": "As a tonic for nerves and the brain",
"d": "To reduce inflammation"
},
"correct": "c"
},
"3": {
"mcq": "Why is it recommended to take homeopathic medications under the supervision of a doctor?",
"options": {
"a": "To avoid side effects",
"b": "To ensure correct dosage",
"c": "To prevent bacterial growth",
"d": "To speed up recovery"
},
"correct": "b"
},
"4": {
"mcq": "What caution is advised for pregnant women regarding homeopathic medications?",
"options": {
"a": "They should take double the dosage",
"b": "They should avoid taking any homeopathic medication",
"c": "They should consult a healthcare provider before taking any medication

In [42]:
quiz = response.get("quiz")

In [43]:
quiz = json.loads(quiz)
quiz

{'1': {'mcq': 'What is Kali phosphoricum?',
  'options': {'a': 'A homeopathic medication made from potassium phosphate',
   'b': 'A modern medicine for stress relief',
   'c': 'A type of surgical procedure',
   'd': 'An antibiotic drug'},
  'correct': 'a'},
 '2': {'mcq': 'What is the main usage of Kali phosphoricum in homeopathy?',
  'options': {'a': 'Treatment of skin conditions',
   'b': 'Tonic for nerves and the brain',
   'c': 'Treatment of heart diseases',
   'd': 'Treatment of respiratory infections'},
  'correct': 'b'},
 '3': {'mcq': 'Which of the following conditions can Kali phosphoricum be prescribed for?',
  'options': {'a': 'High blood pressure',
   'b': 'Chronic fatigue',
   'c': 'Diabetes',
   'd': 'Broken bones'},
  'correct': 'b'},
 '4': {'mcq': 'What caution should be taken when consuming Kali phosphoricum?',
  'options': {'a': 'Avoid chewing tobacco or drinking alcohol',
   'b': 'Take it with any type of beverage',
   'c': 'Consume it with high-fat foods',
   'd': 'Mi

In [44]:
quiz.items()


dict_items([('1', {'mcq': 'What is Kali phosphoricum?', 'options': {'a': 'A homeopathic medication made from potassium phosphate', 'b': 'A modern medicine for stress relief', 'c': 'A type of surgical procedure', 'd': 'An antibiotic drug'}, 'correct': 'a'}), ('2', {'mcq': 'What is the main usage of Kali phosphoricum in homeopathy?', 'options': {'a': 'Treatment of skin conditions', 'b': 'Tonic for nerves and the brain', 'c': 'Treatment of heart diseases', 'd': 'Treatment of respiratory infections'}, 'correct': 'b'}), ('3', {'mcq': 'Which of the following conditions can Kali phosphoricum be prescribed for?', 'options': {'a': 'High blood pressure', 'b': 'Chronic fatigue', 'c': 'Diabetes', 'd': 'Broken bones'}, 'correct': 'b'}), ('4', {'mcq': 'What caution should be taken when consuming Kali phosphoricum?', 'options': {'a': 'Avoid chewing tobacco or drinking alcohol', 'b': 'Take it with any type of beverage', 'c': 'Consume it with high-fat foods', 'd': 'Mix it with other medications'}, 'cor

In [47]:
quiz_table_data = []
for key,value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option,option_value in value["options"].items()
        ]
    )

    correct = value["correct"]
    quiz_table_data.append({"MCQ":mcq, "Choices":options, "Correct": correct})

In [48]:
quiz_table_data

[{'MCQ': 'What is Kali phosphoricum?',
  'Choices': 'a: A homeopathic medication made from potassium phosphate | b: A modern medicine for stress relief | c: A type of surgical procedure | d: An antibiotic drug',
  'Correct': 'a'},
 {'MCQ': 'What is the main usage of Kali phosphoricum in homeopathy?',
  'Choices': 'a: Treatment of skin conditions | b: Tonic for nerves and the brain | c: Treatment of heart diseases | d: Treatment of respiratory infections',
  'Correct': 'b'},
 {'MCQ': 'Which of the following conditions can Kali phosphoricum be prescribed for?',
  'Choices': 'a: High blood pressure | b: Chronic fatigue | c: Diabetes | d: Broken bones',
  'Correct': 'b'},
 {'MCQ': 'What caution should be taken when consuming Kali phosphoricum?',
  'Choices': 'a: Avoid chewing tobacco or drinking alcohol | b: Take it with any type of beverage | c: Consume it with high-fat foods | d: Mix it with other medications',
  'Correct': 'a'},
 {'MCQ': 'Is Kali phosphoricum approved by the U.S. Food a

In [49]:
quiz = pd.DataFrame(quiz_table_data)
quiz

,MCQ,Choices,Correct
0,What is Kali phosphoricum?,a: A homeopathic medication made from potassiu...,a
1,What is the main usage of Kali phosphoricum in...,a: Treatment of skin conditions | b: Tonic for...,b
2,Which of the following conditions can Kali pho...,a: High blood pressure | b: Chronic fatigue | ...,b
3,What caution should be taken when consuming Ka...,a: Avoid chewing tobacco or drinking alcohol |...,a
4,Is Kali phosphoricum approved by the U.S. Food...,"a: Yes, for all age groups | b: No, it has not...",b
5,Who should consult a healthcare provider befor...,a: Only pregnant women | b: Children and adole...,b
6,What is the caution given by the U.S. Food and...,a: They are all safe for consumption | b: They...,b
7,What is the recommendation regarding conventio...,a: Homeopathic products can replace convention...,b
8,What should be done before starting homeopathi...,a: Consult a healthcare professional | b: Star...,a
9,What complications can arise from unsterile ma...,a: No complications | b: Minor skin irritation...,c


In [50]:
quiz.to_csv("biology.csv",index=False)

In [51]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'03_17_2024_15_51_41'